In [66]:
import numpy as np
import scipy.special
import pandas as pd

from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, ColumnDataSource
from ipywidgets import interact
from bokeh.io import show, push_notebook, output_notebook
output_notebook()

p1 = figure(title="",tools="save",
            background_fill_color="#E8DDCB")

#read in fakes
fakes  = pd.read_csv('../data/FakeBackground.csv',names = ['Lep1Pt','Lep2Pt','Jet1Pt','Jet2Pt','HT','nConst','DilepMass','weight','nTL'])
#add smearing for visualization                                                                                                                                                                                    
fakes['Smear'] = np.random.normal(0,0.3,len(fakes))
fakes['nConstSmeared'] = fakes['nConst']+fakes['Smear']
#define column data source for fakes
fakeSource = ColumnDataSource(data=dict(HT=[], nConst=[],weights=[]))

#define selection for fakes
def getFakesData(ht,nc):
    #selected = fakes[(fakes['HT'] >= ht) & (fakes['nConstSmeared'] >= nc)]
    selected = fakes[
        (fakes.HT >= ht) &
        (fakes.nConstSmeared >= nc)
    ]
    return selected

#read in signal
signal  = pd.read_csv('../data/Signal.csv',names = ['Lep1Pt','Lep2Pt','Jet1Pt','Jet2Pt','HT','nConst','DilepMass','weight','nTL'])
signalHT = signal.HT
#add smearing for visualization                                                                                                                                                                                    
signal['Smear'] = np.random.normal(0,0.3,len(signal))
signal['nConstSmeared'] = signal['nConst']+signal['Smear']
signalNConst = signal.nConstSmeared

def getSignalData(ht,nc):
    selected = signal[
        (signal.HT >= ht) &
        (signal.nConst >= nc)
    ]
    return selected

realBkg  = pd.read_csv('../data/RealBackground.csv',names = ['Lep1Pt','Lep2Pt','Jet1Pt','Jet2Pt','HT','nConst','DilepMass','weight','nTL'])
realBkgHT = realBkg.HT
#add smearing for visualization                                                                                                                                                                                    
realBkg['Smear'] = np.random.normal(0,0.3,len(realBkg))
realBkg['nConstSmeared'] = realBkg['nConst']+realBkg['Smear']
#define column data source for fakes
realSource = ColumnDataSource(data=dict(HT=[], nConst=[],weights=[]))
realBkgNConst = realBkg.nConstSmeared

#define selection for fakes
def getRealData(ht,nc):
    #selected = fakes[(fakes['HT'] >= ht) & (fakes['nConstSmeared'] >= nc)]
    selected = realBkg[
        (realBkg.HT >= ht) &
        (realBkg.nConstSmeared >= nc)
    ]
    return selected

f = p1.circle(x='HT',y='nConst',source=fakeSource, legend = 'Fake Background',size=1)
s = p1.triangle(x='HT',y='nConst',source=realSource, legend = 'Real Background', color='red',size=1)
r = p1.asterisk(signalHT,signalNConst, legend = 'Signal', color='green',size=1)

p1.legend.location = "top_right"
p1.xaxis.axis_label = 'Total Event Energy (GeV)'
p1.yaxis.axis_label = 'Number of Particles in Event'

plot = gridplot(p1, ncols=1, plot_width=400, plot_height=400, toolbar_location=None)
t = show(plot,notebook_handle=True)

def update(HT=0,nConstituents=0):
        fakesDF = getFakesData(HT,nConstituents)
        fakeSource.data = dict(HT = fakesDF['HT'], nConst = fakesDF['nConstSmeared'])
        nll = len(fakesDF[fakesDF.nTL==0])
        nlt = len(fakesDF[fakesDF.nTL==2])
        ntt = len(fakesDF[fakesDF.nTL==3])
        realDF = getRealData(HT,nConstituents)
        realSource.data = dict(HT = realDF['HT'], nConst = realDF['nConstSmeared'])
        push_notebook(handle=t)


Loading BokehJS ...

In [67]:
interact(update,HT=(0,1200,50),nConstituents=(0,13,1))

In [71]:


def getNonPromptPred(ht,nc):
    p1 = figure(title="Distribution for Fake Background Prediction",
            background_fill_color="#E8DDCB")
    df = getFakesData(ht,nc)
    nll_obs = len(df[df.nTL==0])
    nlt_obs = len(df[df.nTL==1])
    ntt_obs = len(df[df.nTL==3])

    weight_ll = w_ll(0.202,0.839)
    weight_lt = w_lt(0.202,0.839)
    weight_tt = w_tt(0.202,0.839)

    n_ll = np.random.poisson(nll_obs, 10000)
    n_lt = np.random.poisson(nlt_obs, 10000)
    n_tt = np.random.poisson(ntt_obs, 10000)

    n_np = []

    for nll, nlt, ntt in zip(n_ll,n_lt,n_tt):
        n_np.append( nll*weight_ll + nlt*weight_lt + ntt*weight_tt)

    mean = np.mean(n_np)
    sigma = np.std(n_np)

    hist, edges = np.histogram(n_np, density=False, bins=50)


    p1.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
        fill_color="#036564", line_color="#033649")

    p1.legend.location = "top_left"
    p1.xaxis.axis_label = 'Number of Fake Background Events Predicted'
    p1.yaxis.axis_label = ''
    meantext = Label(x=230, y=350, x_units='screen', y_units='screen',
                     text='Prediction: %.1f Evts' %mean, render_mode='canvas',
                    border_line_color='black',background_fill_color='white')

    sigmatext = Label(x=230, y=330, x_units='screen', y_units='screen',
                 text='Uncert: +/- %.1f Evts' %sigma, render_mode='canvas',
                     border_line_color='black',background_fill_color='white')
    p1.add_layout(meantext)
    p1.add_layout(sigmatext)
    #make figure to hold simple text result
    sigdf = getSignalData(ht,nc)
    sig = len(sigdf) #just number of entries
    sig = sig* (12.9*21*.8/300000) #weighting for luminosity
    res = 'Yes' if (sig > 2*sigma) else 'No'
    sigtext = Label(x=100, y=275, x_units='screen', y_units='screen',
                     text='Signal Events: %.1f Events' %sig, render_mode='canvas')
    restext = Label(x=100, y=225, x_units='screen', y_units='screen',
                     text='Sensitive? : %.1s' %res, render_mode='canvas')
    p2 = figure(title="Analysis Results")
    p2.add_layout(sigtext)
    p2.add_layout(restext)
    return p1,p2

def updatePred(EventEnergy=0,nParticles=0):
    p1,p2 = getNonPromptPred(EventEnergy,nParticles)
    show(gridplot(p1,p2,ncols=2,plot_width=400,plot_height=400),notebook_handle = True)
    push_notebook()

In [72]:
interact(updatePred,EventEnergy=(0,1200,100),nParticles=(0,13,1))

<function __main__.updatePred>